## Что такое категориальный признак, как с ним бороться и зачем

**Категориальный признак** – это признак, который принимает значение из какого-то конечного набора (категории).
Например, 
 - категория "пол",  значения: М, Ж
 - категория "страна",  значения: Россия, США, Германия и т.д.
 - категория "уровень образования",  значения: среднее общее, среднее профессиональное, бакалавриат, специалитет, магистратура
 - др.
 
Также к категориальным можно отнести признаки, которые принимают числовой вид, но являются неизмеримыми, над ними не следует осуществлять арифметические операции. По своей сути это также значение из какого-то конечного набора, но сложность в том, что значения в этом наборе принимают числовой вид, что затрудняет процесс их обнаружения (далее будем называть их **неявными категориалными признаками**).
Например, 
- номер мобильного телефона абонента: представляет собой последовательность чисел, которую можно представить в виде одного дискретного числа. При этом с данным признаком некорректно осуществлять операции сложения/вычитания/деления/умножения, т.к. это не количественный признак. Логичнее будет определить оператора сотовой связи и в дальнейшем использовать данную информацию 
- почтовый индекс 
- др.
 
 
Среди категориальных признаков можно выделить порядковые признаки - признаки, которые можно проранжировать. 
Например, уровень образования. 
 
Категориальные признаки, наряду с вещественными признаками, содержат в себе характеристики объекта исследования, поэтому их необходимо использовать при моделировании. Проблема лишь в том, что **большинство методов машинного обучения способны обработать только признаки в виде вещественных векторов**, поэтому возникает потребность в изменении представления категориальных признаков. Но, обо всем по порядку ... 

### Как обнаружить категориальные признаки?

Самым простой способ - прочитать типы данных (для категориальных признаков характерен строковый тип).

Для выявления неявных категориальных признаков, скорее всего, поможет лишь здравый смысл. Прежде чем использовать какой-либо dataset необходимо ознакомиться с описанием признаков, которое поможет выявить такие признаки. 

В некоторых источниках предлагают обращать внимание на количество уникальных значений. Предполагается, что чем меньше уникальных значений, тем выше вероятность того, что мы имеем дело с неявным категориальным признаком. Но, можно привести массу примеров, когда данная проверка упустит нужные нам признаки, поэтому всё-таки самый надежный способ - пробежаться глазками по описанию признаков.

Рассмотрим на примере датасета flights 

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

df = pd.read_csv('flights.csv')

df.head()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
0,2014,1,1,1.0,96.0,235.0,70.0,AS,N508AS,145,PDX,ANC,194.0,1542,0.0,1.0
1,2014,1,1,4.0,-6.0,738.0,-23.0,US,N195UW,1830,SEA,CLT,252.0,2279,0.0,4.0
2,2014,1,1,8.0,13.0,548.0,-4.0,UA,N37422,1609,PDX,IAH,201.0,1825,0.0,8.0
3,2014,1,1,28.0,-2.0,800.0,-23.0,US,N547UW,466,PDX,CLT,251.0,2282,0.0,28.0
4,2014,1,1,34.0,44.0,325.0,43.0,AS,N762AS,121,SEA,ANC,201.0,1448,0.0,34.0


In [2]:
# смотрим типы данных в столбцах DataFrame
df.dtypes

year           int64
month          int64
day            int64
dep_time     float64
dep_delay    float64
arr_time     float64
arr_delay    float64
carrier       object
tailnum       object
flight         int64
origin        object
dest          object
air_time     float64
distance       int64
hour         float64
minute       float64
dtype: object

In [3]:
# выделим названия столбцов в типом данных 'object'
categorical_columns = df.columns[df.dtypes == 'object']
categorical_columns

Index(['carrier', 'tailnum', 'origin', 'dest'], dtype='object')

In [4]:
df_cat = df[categorical_columns].fillna('NAN')

в рассматриваемом датасете такие признаки, как year, month, day тоже являются категориальными.
к ним также применимы нижеописанные способы + возможно создание дополнительных предикторов на основе них (например, время года на основе месяца). 

### Как преобразовать категориальные признаки?

#### Метод №1  LabelEncoder

*Суть:* каждому значению категориального признака ставит в соотвествие какое-то число

*Преимущества метода:*
 - не увеличивает размерность датасета

*Недостатки метода:*
 - при появлении нового значения -> упадет ошибка
 - числовое значение признака может не совпадать со смысловым значением категории (в случае порядковых категориальных признаков) 
 - некорректная интерпретация: каким-то значениям будет присвоено бОльшее число, хотя исходные категории были несопоставимы (нельзя точно определить, кому присвоить меньшее/большее числовое значение)


In [5]:
from sklearn.preprocessing import LabelEncoder

df_cat.apply(LabelEncoder().fit_transform).head()


,carrier,tailnum,origin,dest
0,1,1285,0,1
1,8,167,1,12
2,7,587,0,29
3,8,1427,0,12
4,1,2279,1,1


#### Метод №2  OneHotEncoder / get_dummies

*Суть:* создается n дополнительных бинарных столбцов (по количеству уникальных значений в столбце) = создание дамми-переменных


*Преимущества метода:*
 - при появлении нового значения ошибка не упадет, но при этом значение никак не учтется (+/-)
 - корректная интерпретация, не завышает числовое значение какой-либо из категорий (все в равных условиях): в случае принадлежности к категории - 1, в противном случае - 0 

*Недостатки метода:*
 - значительно увеличивает размерность датасета

In [6]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse=False)

df_cat_oh = pd.DataFrame(onehot_encoder.fit_transform(df[['carrier', 'origin']]))
df_cat_oh.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [7]:
# для получения понятных столбцов
ohe = OneHotEncoder(categories='auto')
feature_arr = ohe.fit_transform(df[['carrier', 'origin']]).toarray()
feature_labels = ohe.categories_

feature_labels = np.array(feature_labels).ravel()

features = pd.DataFrame(feature_arr, columns=list(feature_labels[0]) + list(feature_labels[1]))
features.head()

,AA,AS,B6,DL,F9,HA,OO,UA,US,VX,WN,PDX,SEA
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
# второй способ
df_with_dummies = pd.get_dummies(df, columns=categorical_columns, drop_first=True)
df_with_dummies.head()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,flight,air_time,distance,...,dest_SEA,dest_SFO,dest_SIT,dest_SJC,dest_SLC,dest_SMF,dest_SNA,dest_STL,dest_TPA,dest_TUS
0,2014,1,1,1.0,96.0,235.0,70.0,145,194.0,1542,...,0,0,0,0,0,0,0,0,0,0
1,2014,1,1,4.0,-6.0,738.0,-23.0,1830,252.0,2279,...,0,0,0,0,0,0,0,0,0,0
2,2014,1,1,8.0,13.0,548.0,-4.0,1609,201.0,1825,...,0,0,0,0,0,0,0,0,0,0
3,2014,1,1,28.0,-2.0,800.0,-23.0,466,251.0,2282,...,0,0,0,0,0,0,0,0,0,0
4,2014,1,1,34.0,44.0,325.0,43.0,121,201.0,1448,...,0,0,0,0,0,0,0,0,0,0


#### Метод №3  Target Encoder

*Суть:* каждому значению категориального признака ставит в соотвествие число, характеризующее связь с таргетом
 
*Преимущества метода:*
 - не увеличивает размерность датасета
 - корректная интерпретация: значение признака говорит о корреляции с целевой меткой
 
*Недостатки метода:*
 - есть риск переобучения модели

In [ ]:
pip install category_encoders

In [11]:
import category_encoders as ce
from category_encoders import TargetEncoder

In [14]:
tar_enc = TargetEncoder().fit(df[['carrier', 'origin']], df['dep_delay'])

In [15]:
df_tar_enc = tar_enc.transform(df[['carrier', 'origin']])
df_tar_enc.head()

,carrier,origin
0,2.783932,5.710905
1,2.734850,6.340789
2,9.795162,5.710905
3,2.734850,5.710905
4,2.783932,6.340789


более полную информацию о всех видах кодировки категориальных признаков см.по ссылке ниже

https://contrib.scikit-learn.org/categorical-encoding/